In [11]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import boto3
import base64
from io import BytesIO
from PIL import Image
import os
from dotenv import load_dotenv, find_dotenv

In [12]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
# Encontra o arquivo .env no diretório pai
dotenv_path = find_dotenv()

# Carrega as variáveis de ambiente do arquivo .env encontrado
load_dotenv(dotenv_path)

True

In [13]:
dataset_peoples = pd.read_csv('../data/peoples.csv')
dataset_pets = pd.read_csv('../data/dataset_union.csv')

In [14]:
uuids_peoples = dataset_peoples['ID'].to_list()
ids_pets = dataset_pets['ID'].to_list()

In [15]:
from collections import namedtuple

Pet = namedtuple('Pet', ['id', 'animal_name', 'animal_gender', 'animal_birth_month', 'breed_name', 'borough', 'zip_code', 'type', 'image'])

In [16]:
print(len(dataset_pets))

134291


In [17]:
dict_people_pet = {}

# garantindo que todo usuário terá um animal
for index, id in tqdm(enumerate(uuids_peoples), total=len(uuids_peoples)):
    pet = dataset_pets.loc[dataset_pets.index[index]]
    dict_people_pet[f"{id}"] = [Pet(id = pet['ID'], animal_name = pet['AnimalName'], animal_gender = pet['AnimalGender'], animal_birth_month = pet['AnimalBirthMonth'], breed_name = pet['BreedName'], borough = pet['Borough'],  zip_code = str(pet['ZipCode']), type = pet['Type'], image = pet['Image'])]
    if index in dataset_pets.index:
        dataset_pets = dataset_pets.drop(index)

 11%|█         | 2755/24500 [01:44<13:42, 26.43it/s]


KeyboardInterrupt: 

In [ ]:
uuids_peoples = dataset_peoples['ID'].to_list()


In [ ]:
# distribuir aleatoriamente os animais para os usuarios

count_pets = len(ids_pets)
count_users = len(uuids_peoples)


count_pets_distributeds = 0
tqdm_instance = tqdm(total=count_pets)

dict_people_pet_final = {}


while count_pets_distributeds <= count_pets or len(uuids_peoples) <= 0:
    people_random = np.random.choice(uuids_peoples)
    quantity_pets = np.random.randint(1, 8)

    for _ in range(quantity_pets):
        ids_pets = dataset_pets['ID'].to_list()
        random_index = np.random.choice(ids_pets)
        if random_index in dataset_pets.index:
            pet = dataset_pets.loc[random_index]
            dict_people_pet[people_random].append(Pet(id = pet['ID'], animal_name = pet['AnimalName'], animal_gender = pet['AnimalGender'], animal_birth_month = pet['AnimalBirthMonth'], breed_name = pet['BreedName'], borough = pet['Borough'],  zip_code = str(pet['ZipCode']), type = pet['Type'], image = pet['Image']))
            dataset_pets = dataset_pets.drop(random_index)
            tqdm_instance.update(1)

    dict_people_pet_final[people_random] = dict_people_pet.pop(people_random)
    uuids_peoples.remove(people_random)

    count_pets_distributeds += quantity_pets

  1%|          | 1424/134291 [00:26<41:12, 53.73it/s]

KeyError: '79c83b55-4792-4995-b9fa-eb42ae601796'

In [ ]:
print(dict_people_pet_final)

In [ ]:
print(os.getenv('AWS_ACCESS_KEY_ID'))

In [ ]:
session = boto3.Session(
    aws_access_key_id       = os.getenv('AWS_ACCESS_KEY_ID'),  
    aws_secret_access_key   = os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name             = os.getenv('AWS_DEFAULT_REGION'), 
    aws_session_token       = os.getenv('AWS_ACCESS_TOKEN')
)

In [ ]:
s3 = session.resource('s3')

In [ ]:
def directory_exists(bucket_name, directory_name):
    try:
        s3.head_object(Bucket=bucket_name, Key=directory_name+'/')
        return True
    except:
        return False
    
def is_base64(text):
    try:
        base64.b64decode(text)
        return True
    except:
        return False

In [ ]:
bucket_name = 'pilha-nuvem-tcc-sptech-bucket'

for user_id, pets in dict_people_pet.items():
    user_folder = f"s3://{bucket_name}/train/{user_id}"
    # cria diretório para o usuário
    if directory_exists(bucket_name = bucket_name, directory_name = f"/train/{user_id}"):
        continue

    s3.Bucket(bucket_name).put_object(Key=user_id + "/", Body="")
    for pet in pets:
        # converte a imagem em base64 em uma imagem JPG
        if not is_base64(pet.image):
            continue
        
        img_data = base64.b64decode(pet.image.split(',')[1])
        img = Image.open(BytesIO(img_data))
        # cria diretório para o pet dentro do diretório do usuário
        pet_folder = user_folder + f"/{pet.id}"
        s3.Bucket("bucket_name").put_object(Key=pet_folder + "/", Body="")
        # salva a imagem do pet no diretório do pet
        img.save(pet_folder + "/pet_image.jpg", "JPEG")